<a href="https://colab.research.google.com/github/cheatcodesInc/Fake-news-detector/blob/main/getWikipediaFacts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers
!pip install wikipedia
# !wget -c "https://figshare.com/ndownloader/files/13774556"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=b03007c484a54f4ca1e68cb2ab048cbc6574d8ec08c3dc41e069d26737c2ce06
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [ ]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from pprint import pprint
from nltk.util import ngrams
import wikipedia as wiki
import gensim
import re
import nltk
from nltk import tokenize
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


model = SentenceTransformer('bert-base-nli-mean-tokens')
nlp = en_core_web_sm.load()
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Use n-grams for search

In [ ]:
def keywords(sentence):

    doc = nlp(sentence)

    results = ([(X.text, X.label_)[0] for X in doc.ents])
    n_gram=[]

    for i in range(2, 5):
        n_gram.extend(ngrams(sentence.split(), i))

    results.extend([' '.join(i) for i in n_gram])
    return list(set(results))

In [ ]:
def clean_wiki_text(text):
    text = re.sub(r'==.+==', '.', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\t', ' ', text)
    text = re.sub(r'\[[0-9]+\]', ' ', text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r'\. \.', '.', text)
    return text

In [ ]:
def content(claim, results):
    sentences = []
    found=[]
    for i in results:
        try :
            current_page = wiki.page(i)
            if current_page not in found:
                found.append(current_page)
        except :
            continue
    titles=[i.title for i in found]
    titles=[i[0] for i in topNSimilar(claim, titles)]
    for i in found:
        if i.title not in titles:
            found.remove(i)

    for i in found:
        current_content = i.content
        sentences.extend(tokenize.sent_tokenize(clean_wiki_text(current_content)))
    return sentences

In [ ]:
def topNSimilar(claim, sentList, n=5):
    distList=[]
    for i in sentList:
        document_term_matrix = TfidfVectorizer().fit_transform([i, claim])
        dist = document_term_matrix * document_term_matrix.transpose()
        distance = dist.toarray()[0][1]

        if len(distList)<=n:
            distList.append([i, distance])
        else:
            distList=sorted(distList, key=lambda x: x[1], reverse=True)
            if distance>distList[-1][1]:
                distList.pop()
                distList.append([i, distance])
    return sorted(distList, key=lambda x: x[1], reverse=True)

In [ ]:
claim = 'Donald Trump is the president of the united states'

In [ ]:
response = topNSimilar(claim, content(claim, keywords(claim)), 5)

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [ ]:
def truthfulness(claim, response):
    

In [ ]:
# import nltk
# 


In [ ]:
res=content(claim, keywords(claim))

In [ ]:
res[1]

['Vande Mataram',
 'Written By',
 'Bankim Chandra Chatterjee',
 'It Was Written',
 'Sarat Chandra Chattopadhyay']

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
for i in response:
    sim_arr=cosine_similarity( [model.encode(claim)], [model.encode(i[0])] )[0][0]
    print(i[0], sim_arr)

The president of the United States (POTUS) is the head of state and head of government of the United States of America. 0.76244354
The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces. 0.7018652
The president has the power to nominate federal judges, including members of the United States courts of appeals and the Supreme Court of the United States. 0.62361306
This is the only component in the inauguration ceremony mandated by the Constitution: I do solemnly swear (or affirm) that I will faithfully execute the Office of President of the United States, and will to the best of my ability, preserve, protect, and defend the Constitution of the United States. 0.5327108
The position of the United States as the leading member of NATO, and the country's strong relationships with other wealthy or democratic nations like those comprising the European Union, have led to the moniker that the president is the "leader of

In [ ]:
def topNSimilar(claim, sentList, n=5):
    distList=[]
    for i in sentList:
        document_term_matrix = TfidfVectorizer().fit_transform([i, claim])
        dist = document_term_matrix * document_term_matrix.transpose()
        distance = dist.toarray()[0][1]

        if len(distList)<=n:
            distList.append([i, distance])
        else:
            distList=sorted(distList, key=lambda x: x[1], reverse=True)
            if distance>distList[-1][1]:
                distList.pop()
                distList.append([i, distance])
    return sorted(distList, key=lambda x: x[1], reverse=True)

In [ ]:
claim = 'Vande Mataram was written by Bankim Chandra Chattopadhyay'

In [ ]:
topNSimilar(claim, content(claim, keywords(claim)), 5)

[['If an anthem is defined as consisting of both a melody and lyrics, then the oldest national anthem in use today is the national anthem of the Netherlands, the Wilhelmus.',
  0.47482540034883847],
 ['One of these is the "Marcha Real", the national anthem of Spain.',
  0.46348639791287216],
 ['9 is the official anthem of the European Union and of the Council of Europe.',
  0.41088847165649467],
 ['"Esta É a Nossa Pátria Bem Amada" is the national anthem of Guinea-Bissau and was also the national anthem of Cape Verde until 1996.',
  0.4074540453346758],
 ['The national chair is the leading volunteer of the National Executive Board of the Boy Scouts of America, a position comparable to the chairman of a board of directors.',
  0.4049238481992085],
 ['The national anthem of Pakistan, the "Qaumi Taranah", is unique in that it is entirely in Farsi (Persian) with the exception of one word which is in Urdu, the national language.',
  0.3975677672553976]]

In [ ]:
result=topNSimilar(claim, res[0])

In [ ]:
titles= [i.title for i in res[1]]

In [ ]:
titles

['Death of Subhas Chandra Bose',
 'Subhas Chandra Bose',
 'Aviation accidents and incidents',
 'Ritchie Valens',
 'Dior',
 'Bachelor of Arts',
 'Lynyrd Skynyrd plane crash']

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
data = res[0]

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
max_epochs = 80
vec_size = 30
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=2,
                dm =1,
                epochs= max_epochs)
  
model.build_vocab(tagged_data)



/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [ ]:
model.train(tagged_data, total_examples=model.corpus_count, epochs=max_epochs)

test_data = word_tokenize(claim.lower())
v1 = model.infer_vector(test_data)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
# print(similar_doc)


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
# print(model.docvecs['1'])


In [ ]:
for i in range(5):
    print(data[int(similar_doc[i][0])])

Chattopadhyay was very intereste in recent events in Indian and Bengali history, particularly the Revolt of 1857 and the previous century's Sanyasi Rebellion.
The poem was published in Chattopadhyay's book Anandamath (pronounced Anondomôţh in Bengali) in 1882, which is set in the events of the Sannyasi Rebellion.
from Bengali by Surendranath Tagore.
Durgeshnandini, his first Bengali romance and the first ever novel in Bengali, was published in 1865.
His first attempt was a novel in Bengali submitted for a declared prize.


In [ ]:
[print(i[0]) for i in result]

It is generally believed that the concept of Vande Mataram came to Bankim Chandra Chattopadhyay when he was still a government official, around 1876.
Media related to Bankim Chandra Chattopadhyay at Wikimedia Commons Quotations related to Bankim Chandra Chatterjee at Wikiquote Works by Bankim Chandra Chatterji at Project Gutenberg Works by or about Bankim Chandra Chatterjee at Internet Archive Works by Bankim Chandra Chatterjee at LibriVox (public domain audiobooks) https://en.banglapedia.org/index.php/Chattopadhyay,_Bankimchandra
It Was Written was generally well received by critics.
The first translation of Bankim Chandra Chattopadhyay's novel Anandamath, including the poem Vande Mataram, into English was by Nares Chandra Sen-Gupta, with the fifth edition published in 1906 titled "The Abbey of Bliss".Here is the translation in prose of the above two stanzas rendered by Sri Aurobindo Ghosh.
Written By at IMDb
Matangini Hazra's last words as she was shot to death by the Crown police we

[None, None, None, None, None, None]